In [ ]:
import time as time
import pandas as pd
import os

from selenium import webdriver 

In [ ]:
# Helper function to save data from memory to disk
def appendToFile(filepath, df):
    header = True
    # If file exist then don't add the header
    if os.path.isfile(filePath):
        header = False
        
    df.to_csv(filePath, mode='a', header=header, index=False)

In [ ]:
url = 'http://cpsnsphysiciansearch.azurewebsites.net/SearchResults.aspx'

# Create an empty global list
df = pd.DataFrame()

In [ ]:
option = webdriver.ChromeOptions()
option.add_argument('-incognito')

browser = webdriver.Chrome(executable_path='../chromedriver', chrome_options=option)

In [ ]:
# Go to search page
browser.get(url)

# Wait for the page to load
time.sleep(1)

# Click on the view options button
browser.find_element_by_xpath("//button[@class='btn btn-default dropdown-toggle']").click()

# Click on all in the drop down menu
browser.find_element_by_xpath("//a[contains(text(), 'All')]").click()

# Wait 5 seconds for all the rows to laod
time.sleep(3)

# Grab the table that has the doctors 
table = browser.find_element_by_xpath("//table[@id='grid-basic']")

# Grab the a tags to all the doctor profiles
aList = table.find_elements_by_tag_name('a')

# Grab the urls from each a tag at the same time splice out the headers
urls = list(map(lambda x: x.get_attribute('href'), aList[2:]))

#Loop through each link
for aLink in urls:
    # Go to link
    browser.get(aLink)
    time.sleep(2)
    
    # If there is any info try to scrape the data
    try:
        # Scrape the info
        licence = browser.find_element_by_id('MainContent_frmlicencetype').get_attribute("value")
        address = browser.find_element_by_id('MainContent_frmbusinessaddress').get_attribute("value")
        firstName = browser.find_element_by_id('MainContent_frmgivenname').get_attribute("value")
        lastName = browser.find_element_by_id('MainContent_frmsurname').get_attribute("value")
        licenceNumber = int(aLink.split('=')[1].strip('0'))
        
        # Package the info
        data = {
            'Licence Status': licence,
            'Address': address,
            'First Name': firstName,
            'Last Name': lastName,
            'Licence Number': licenceNumber
        }
        
        # Append it to the main dataframe
        df = df.append(data, ignore_index=True)
    except:
        print('failed')
        
    
    # Free up RAM space
    del licence, address, firstName, lastName, licenceNumber

In [ ]:
df.to_csv('./csv/NS Physicians.csv', index=False)